# Ensemble Learning

## Initial Imports

In [196]:
import warnings
warnings.filterwarnings('ignore')

In [197]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import calendar
from sklearn.preprocessing import LabelEncoder

In [198]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

## Read the CSV and Perform Basic Data Cleaning

In [199]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [177]:
#We remove year part from issue_d and next_pymnt_d columns as year is same for all the rows which is 2019

df['issue_d'] = df['issue_d'].str[0:3]
df['next_pymnt_d'] = df['next_pymnt_d'].str[0:3]

In [178]:
# Months dictionary
months_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
months_num

{'Jan': 1,
 'Feb': 2,
 'Mar': 3,
 'Apr': 4,
 'May': 5,
 'Jun': 6,
 'Jul': 7,
 'Aug': 8,
 'Sep': 9,
 'Oct': 10,
 'Nov': 11,
 'Dec': 12}

In [179]:
# Create the LabelEncoder instance
label_encoder = LabelEncoder()

# Fitting and encoding the columns with the LabelEncoder
for col in df[
    ['verification_status', 
     'pymnt_plan', 
     'initial_list_status', 
     'application_type', 
     'hardship_flag', 
     'debt_settlement_flag']]:
    label_encoder.fit(df[col])
    df[col] = label_encoder.transform(df[col])

In [180]:
# Encoding the `home ownership` column
df = pd.get_dummies(df, columns=['home_ownership'])
df.head()

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,1,Mar,low_risk,0,27.24,0.0,...,65687.0,38199.0,2000.0,61987.0,0,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,2,Mar,low_risk,0,20.23,0.0,...,271427.0,60641.0,41200.0,49197.0,0,0,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,2,Mar,low_risk,0,24.26,0.0,...,60644.0,45684.0,7500.0,43144.0,0,0,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,2,Mar,low_risk,0,31.44,0.0,...,99506.0,68784.0,19700.0,76506.0,0,0,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,0,Mar,low_risk,0,18.76,0.0,...,219750.0,25919.0,27600.0,20000.0,0,0,0,1,0,0


In [181]:
# Save the file for forthcoming usage
file_path = Path("Resources/df_encoded.csv")
df.to_csv(file_path, index=False)
df.head()

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,1,Mar,low_risk,0,27.24,0.0,...,65687.0,38199.0,2000.0,61987.0,0,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,2,Mar,low_risk,0,20.23,0.0,...,271427.0,60641.0,41200.0,49197.0,0,0,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,2,Mar,low_risk,0,24.26,0.0,...,60644.0,45684.0,7500.0,43144.0,0,0,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,2,Mar,low_risk,0,31.44,0.0,...,99506.0,68784.0,19700.0,76506.0,0,0,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,0,Mar,low_risk,0,18.76,0.0,...,219750.0,25919.0,27600.0,20000.0,0,0,0,1,0,0


## Split the Data into Training and Testing

In [200]:
# Create our features
X = df.drop(columns=["loan_status", 
                     "home_ownership",
                     "verification_status",
                     "issue_d","pymnt_plan",
                     "initial_list_status",
                    "hardship_flag",
                    "debt_settlement_flag",
                    "next_pymnt_d",
                    "application_type"])
# Create our target
y = df["loan_status"]


In [201]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [202]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [203]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 76)

In [204]:
y_train.shape

(51612,)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [205]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [206]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [207]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[-0.64978133, -0.30327574, -0.52669276, ..., -0.09170382,
        -0.75733047, -0.03294137],
       [ 0.42378352, -0.42616892,  0.71269346, ..., -0.44760726,
         1.3046767 , -0.50474625],
       [ 1.79013878, -0.82400957,  2.19335392, ..., -1.04470431,
         0.438036  , -0.57928896],
       ...,
       [ 0.81417074, -0.36368086,  1.17925683, ..., -0.36245362,
         0.40441632, -0.44272156],
       [-0.25939411, -0.42616892, -0.08220905, ..., -0.20693308,
        -0.70129766,  0.15147531],
       [-0.16179731, -1.196855  , -0.05849257, ..., -0.40364526,
         0.1242523 , -0.46649022]])

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [208]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_model = brf_model.fit(X_train_scaled, y_train)

In [209]:
# Calculated the balanced accuracy score
brf_predictions = brf_model.predict(X_test_scaled)
brf_acc_score = balanced_accuracy_score(y_test, brf_predictions)
print(f"The balanced accuracy score is {brf_acc_score}")

The balanced accuracy score is 0.7113181929890295


In [210]:
# Display the confusion matrix
brf_cm = confusion_matrix(y_test, brf_predictions)
brf_cm_df = pd.DataFrame(
    brf_cm, index=["Actual high_risk", "Actual low_risk "], columns=["Predicted high_risk", "Predicted low_risk"]
)
brf_cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,51,36
Actual low_risk,2800,14318


In [211]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, brf_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.59      0.84      0.03      0.70      0.48        87
   low_risk       1.00      0.84      0.59      0.91      0.70      0.50     17118

avg / total       0.99      0.84      0.59      0.91      0.70      0.50     17205



In [212]:
# List the features sorted in descending order by feature importance
feature_importance = pd.DataFrame(brf_model.feature_importances_,
                                   index = X.columns,
                                    columns=['Feature Importance']).sort_values('Feature Importance', ascending = False)
feature_importance[:10]

,Feature Importance
total_rec_int,0.075948
last_pymnt_amnt,0.070213
total_rec_prncp,0.064822
total_pymnt_inv,0.059630
total_pymnt,0.049460
int_rate,0.030904
dti,0.021827
mths_since_recent_inq,0.017952
tot_hi_cred_lim,0.017880
max_bal_bc,0.017454


### Easy Ensemble Classifier

In [213]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec_model.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [214]:
# Calculated the balanced accuracy score
eec_predictions = eec_model.predict(X_test_scaled)
eec_acc_score = balanced_accuracy_score(y_test, eec_predictions)
print(f"The balanced accuracy score is {eec_acc_score}")

The balanced accuracy score is 0.7511693679973893


In [215]:
# Display the confusion matrix
eec_cm = confusion_matrix(y_test, eec_predictions)
eec_cm_df = pd.DataFrame(
    eec_cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"]
)
eec_cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,62,25
Actual low_risk,3600,13518


In [216]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, eec_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.71      0.79      0.03      0.75      0.56        87
   low_risk       1.00      0.79      0.71      0.88      0.75      0.57     17118

avg / total       0.99      0.79      0.71      0.88      0.75      0.57     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [217]:
print(f"1- Both models have the same balanced acuracy")
print(f"2- Random Forest Clasifier")
print(f"3-Easy Ensemble")
print(f"4-Top three features are: 1)total_rec_prncp 2)total_rec_int 3)last_pymnt_amnt ")

1- Both models have the same balanced acuracy
2- Random Forest Clasifier
3-Easy Ensemble
4-Top three features are: 1)total_rec_prncp 2)total_rec_int 3)last_pymnt_amnt 
